In [1]:
import psycopg2

In [11]:
def connect_to_database():
    # Establish a connection to the PostgreSQL database
    conn = psycopg2.connect(
        database='postgres',
        user='postgres',
        password='Karthick@09',
        host='localhost'
    )
    return conn

In [15]:
def enable_pg_qualstats(conn):
    # Create a cursor object and enable the pg_qualstats extension
    cursor = conn.cursor()
    cursor.execute('CREATE EXTENSION IF NOT EXISTS pg_stat_statements;')
    conn.commit()
    cursor.close()

In [4]:
def enable_hypopg(conn):
    # Create a cursor object and enable the hypopg extension
    cursor = conn.cursor()
    cursor.execute('CREATE EXTENSION IF NOT EXISTS hypopg;')
    conn.commit()
    cursor.close()

In [5]:
def get_query_stats(conn):
    # Retrieve query statistics from the pg_stat_statements view
    cursor = conn.cursor()
    cursor.execute('SELECT query, calls, total_time FROM pg_stat_statements ORDER BY total_time DESC;')
    stats = cursor.fetchall()
    cursor.close()
    return stats

In [6]:
def generate_index_suggestions(conn):
    # Generate index suggestions based on query statistics using hypopg
    cursor = conn.cursor()
    cursor.execute('SELECT hypopg_gen_indexrecommendation();')
    suggestions = cursor.fetchall()
    cursor.close()
    return suggestions

In [7]:
def create_index(conn, table_name, index_definition):
    # Create an index on the specified table using the provided index definition
    create_index_query = f"CREATE INDEX CONCURRENTLY {index_definition} ON {table_name};"
    cursor = conn.cursor()
    cursor.execute(create_index_query)
    conn.commit()
    cursor.close()

In [8]:
def autoindex_based_on_statistics():
    # Connect to the database
    conn = connect_to_database()

    # Enable required extensions
    enable_pg_qualstats(conn)
    enable_hypopg(conn)

    # Retrieve query statistics
    stats = get_query_stats(conn)

    # Generate index suggestions based on query statistics
    suggestions = generate_index_suggestions(conn)

    # Extract table and index information from the suggestions
    for suggestion in suggestions:
        table_name = suggestion[0].split(' ')[2]  # Extract table name from suggestion
        index_definition = suggestion[1].split(' ')[3:]  # Extract index definition from suggestion
        create_index(conn, table_name, ' '.join(index_definition))

    # Close the database connection
    conn.close()

In [16]:
autoindex_based_on_statistics()

FeatureNotSupported: extension "hypopg" is not available
DETAIL:  Could not open extension control file "C:/MyApps/PostgreSQL/15/share/extension/hypopg.control": No such file or directory.
HINT:  The extension must first be installed on the system where PostgreSQL is running.
